# More advanced concepts: Parallel computation and caching

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

The aim of this notebook is:

  - to explain how parallel computation works within scikit-learn
  - how to cache certain computations to save computation time.

For this tutorial we will rely essentially on the [joblib package](https://joblib.readthedocs.io/).

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
import os
from urllib.request import urlretrieve

url = ("https://archive.ics.uci.edu/ml/machine-learning-databases"
       "/adult/adult.data")
local_filename = os.path.basename(url)
if not os.path.exists(local_filename):
    print("Downloading Adult Census datasets from UCI")
    urlretrieve(url, local_filename)

In [3]:
names = ("age, workclass, fnlwgt, education, education-num, "
         "marital-status, occupation, relationship, race, sex, "
         "capital-gain, capital-loss, hours-per-week, "
         "native-country, income").split(', ')    
data = pd.read_csv(local_filename, names=names)

y = data['income']
X_df = data.drop('income', axis=1)

In [4]:
X_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [5]:
y.value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

## Let's construct a full model with a ColumnTransformer

In [7]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

numeric_features = [c for c in X_df
                    if X_df[c].dtype.kind in ('i', 'f')]
categorical_features = [c for c in X_df
                        if X_df[c].dtype.kind not in ('i', 'f')]

pipeline = make_pipeline(
    make_column_transformer(
        (OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        (StandardScaler(), numeric_features),
    ),
    RandomForestClassifier(max_depth=7, n_estimators=300)
)

cv_scores = cross_val_score(pipeline, X_df, y, scoring='roc_auc', cv=5)
print("CV score:", np.mean(cv_scores))

CV score: 0.905253221971409


### How to run things in parallel in scikit-learn: The `n_jobs` parameter

In [8]:
%timeit -n1 -r2 cross_val_score(pipeline, X_df, y, scoring='roc_auc', cv=5)

22.7 s ± 396 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [9]:
%timeit -n1 -r2 cross_val_score(pipeline, X_df, y, scoring='roc_auc', cv=5, n_jobs=-1)

10.7 s ± 3.5 s per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [10]:
%%timeit -n1 -r2

pipeline[-1].set_params(n_jobs=-1) # for the classifier
cv_scores = cross_val_score(pipeline, X_df, y, scoring='roc_auc', cv=5, n_jobs=1) # one task in 1 cpu or one task on many cpu

6.49 s ± 142 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


parallerize better

### How to write your own parallel code with joblib

Let's first look at a simple example:

In [12]:
from joblib import Parallel, delayed
from math import sqrt

delayed(sqrt)(2, x=1)

(<function math.sqrt(x, /)>, (2,), {'x': 1})

In [14]:
import os
import time
def f(x):
    time.sleep(.1)
    print("working on", os.getpid()) #prpcess id
    return 2 * x

print("Main process in ", os.getpid())
Parallel(n_jobs=2)(
    delayed(f)(i) for i in range(5)
)
# [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

Main process in  13212


exception calling callback for <Future at 0x253bdf84610 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "d:\Anaconda3\envs\ima\lib\site-packages\joblib\externals\loky\process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "d:\Anaconda3\envs\ima\lib\multiprocessing\queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_make_function' on <module 'joblib.externals.cloudpickle.cloudpickle' from 'd:\\Anaconda3\\envs\\ima\\lib\\site-packages\\joblib\\externals\\cloudpickle\\cloudpickle.py'>
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\Anaconda3\envs\ima\lib\site-packages\joblib\externals\loky\_base.py", line 26, in _invoke_callbacks
    from concurrent.futures import FIRST_EXCEPTION
  File "d:\Anaconda3\envs\im

Unexpected exception formatting exception. Falling back to standard exception


joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "d:\Anaconda3\envs\ima\lib\site-packages\joblib\externals\loky\process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "d:\Anaconda3\envs\ima\lib\multiprocessing\queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_make_function' on <module 'joblib.externals.cloudpickle.cloudpickle' from 'd:\\Anaconda3\\envs\\ima\\lib\\site-packages\\joblib\\externals\\cloudpickle\\cloudpickle.py'>
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\Anaconda3\envs\ima\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Pacman\AppData\Local\Temp\ipykernel_13212\352706859.py", line 9, in <cell line: 9>
    Parallel(n_jobs=2)(
  F

Let's now do a full cross-validation in parallel:

In [15]:
from sklearn.base import clone

def _fit_score(model, X, y, train_idx, test_idx):
    X_train = X.iloc[train_idx]
    X_test = X.iloc[test_idx]
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]
    model = clone(model)
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import StratifiedKFold

n_jobs = 1

cv = StratifiedKFold(n_splits=5)

scores = Parallel(n_jobs=n_jobs)(delayed(_fit_score)(
    pipeline, X_df, y, train_idx, test_idx
) for train_idx, test_idx in cv.split(X_df, y))

print(scores)

### How about caching?

Something you want to avoid redoing again and again the same computations.
One classical solution to address this is called function [memoization](https://en.wikipedia.org/wiki/Memoization).

joblib offers a very trivial way to do using a simple Python decorator.

In [ ]:
from joblib import Memory

location = '.'
mem = Memory(location, verbose=0)
mem.clear()  # make sure there is not left over cache from previous run

_fit_score_cached = mem.cache(_fit_score)

def evaluate_model():
    scores = Parallel(n_jobs=n_jobs)(delayed(_fit_score_cached)(
        pipeline, X_df, y, train_idx, test_idx
    ) for train_idx, test_idx in cv.split(X_df, y))
    print(scores)

%timeit -n1 -r1 evaluate_model()

In [ ]:
%timeit -n1 -r1 evaluate_model()

Certain transformer objects in scikit-learn have a memory parameter. This allows to cache their computation for example to avoid rerunning the same preprocessing in a grid-search when tuning the classifier or regressor at the end of the pipeline.

To go further you can also look how joblib can be used in combination with [dask-distributed](http://distributed.dask.org/en/stable/) to run computations across different machines or with [dask-jobqueue](http://jobqueue.dask.org/en/latest/) to use a cluster with a queuing system like `slurm`.

In [ ]:
from sklearn.pipeline import Pipeline
Pipeline?